## Data processing for BOW

This notebooks receives the cleaned tweets from the file tweets_nlp_modelling_V2. Which is a second version with stratified sampling.

This file will be processed to obtain the tokens and then apply the BOW function.
Tokens will be grouped by date, in order to include all the tokens for each day.

Libraries and stopwords:

In [1]:
import pandas as pd
from itertools import chain
import nltk
import joblib
from functions.tweets_tokenization import \
    tokenize_tweets, \
    dictionary_tweets, \
    bow_tweets, \
    train_bow_tweets

stopwords = nltk.corpus.stopwords.words(['english'])

Import the data from CSV:

In [2]:
directory = '~/PycharmProjects/tfm_hugopobil'
df = pd.read_csv(f'{directory}/data/sampled_data/tweets_nlp_modelling_v2.csv')
btc_usd_grouped = pd.read_csv(f'{directory}/data/sampled_data/btc_usd_grouped_v2.csv')
df = df.set_index('date_clean')

In [3]:
print(df.shape)
df.head()

(22120, 7)


,tweets,cleaned_tweets,crypto_sentiment,subjectivity,polarity,sentiment,target
date_clean,,,,,,,
2021-02-05,Bitcoin and ETH both have bullish setups for a...,Bitcoin ETH bullish setup move higher BTC woul...,NaN,0.416667,0.350000,neutral,False
2021-02-05,4⃣ 🎙️ Bloomberg LP CryptoOutlook 2021 with ⬇️...,Bloomberg LP CryptoOutlook cryptocurrency bitc...,NaN,0.000000,0.000000,positive,False
2021-02-05,⬇️⬇️ $BTC SELLING PRESSURE ALERT 📉 Price tradi...,BTC SELLING PRESSURE ALERT Price trading aroun...,NaN,0.000000,0.000000,positive,False
2021-02-05,"If hyperinflation does hit again, think of the...",If hyperinflation hit think inflation like flo...,NaN,0.541667,-0.291667,negative,False
2021-02-05,DeriBot Daily Trading Report 5.02.2021 11:42 U...,DeriBot Daily Trading Report UTC Bitcoin Tradi...,NaN,0.000000,0.000000,positive,False


### Tweets tokenization without grouping

In [4]:
df['tokens'] = tokenize_tweets(df.tweets.to_list())

In [5]:
df.sample(n=3).style.set_properties(**{
    'text-align' : 'left',
    'white-space': 'wrap'
})

,tweets,cleaned_tweets,crypto_sentiment,subjectivity,polarity,sentiment,target,tokens
date_clean,,,,,,,,
2022-01-22,"I can't see a big drop, nor a dip, do you? Have checked the market but I can't see any difference one Bitcoin is still worth one Bitcoin. /WnE0lpX0My",I see big drop dip Have checked market I see difference one Bitcoin still worth one Bitcoin WnE lpX My,positive,0.100000,0.150000,neutral,0,"[""can't"", 'see', 'big', 'drop', 'dip', 'checked', 'market', ""can't"", 'see', 'difference', 'one', 'bitcoin', 'still', 'worth', 'one', 'bitcoin', 'wne', '0lpx0my']"
2022-01-21,"🚓 BTC and 14 Others, 🟢 13 🔴 2 🇰🇷 13.98억원 🇺🇸 $1,196,622 🇯🇵 ¥131,435,034 🇪🇺 €1,012,796 🇨🇳 7,725,066元 🎉Upflow volume Now, Upbit(Korea) BTC ETH ATOM SAND STORJ POWR ORBS KNC HBAR KAVA ETC AERGO OMG /wR8kv4Y7Ai",BTC Others Upflow volume Now Upbit Korea BTC ETH ATOM SAND STORJ POWR ORBS KNC HBAR KAVA ETC AERGO OMG wR kv Y Ai,positive,0.000000,0.000000,positive,1,"['btc', 'others', '억원', '066元', 'upflow', 'volume', 'upbit', 'korea', 'btc', 'eth', 'atom', 'sand', 'storj', 'powr', 'orbs', 'knc', 'hbar', 'kava', 'etc', 'aergo', 'omg', 'wr8kv4y7ai']"
2021-02-19,"If you like gold, there are many reasons you should like Bitcoin. Bitcoin BTC cryptocurrency BCH gold silver… /mMn6viAflv",If like gold many reason like Bitcoin Bitcoin BTC cryptocurrency BCH gold silver mMn viAflv,positive,0.500000,0.500000,positive,1,"['like', 'gold', 'many', 'reasons', 'like', 'bitcoin', 'bitcoin', 'btc', 'cryptocurrency', 'bch', 'gold', 'silver', 'mmn', '6viaflv']"


### Dictionary for tokens and Bow processing

In [18]:
dictionary = dictionary_tweets(df['tokens'])

In [7]:
X, doc2bow = bow_tweets(df['tokens'], dictionary)
X_train = train_bow_tweets(X, min_n_words=3)

X_train will not include dates for tokens with less than 3 words. However, I'm still considering wether to use train or normal X.

In [8]:
X_train.shape, X.shape

((21223, 46394), (22120, 46394))

Save the non-grouped tweets tokenization processing to local:
Save doc2bow, dictionary function is applied in each notebook.

In [11]:
df.to_csv(f'{directory}/data/model_data/tweets_tokenize_v2.csv')

In [12]:
joblib.dump(doc2bow, '/Users/hpp/PycharmProjects/tfm_hugopobil/data/model_data/doc2bow_v2.joblib')

['/Users/hpp/PycharmProjects/tfm_hugopobil/data/model_data/doc2bow_v2.joblib']

### Group TOKENS by date:

This will create a dataframe with the accumulation of all tokens with the same date to obtain a grouped tokens.

In [13]:
df_grouped = df.groupby(df.index).agg({'tokens': lambda x: list(chain(*x.to_list()))})

Create dictionary and BOW for grouped tokens:

In [19]:
dictionary_grouped = dictionary_tweets(df['tokens'])

In [20]:
X_grouped, doc2bow_grouped = bow_tweets(df_grouped['tokens'], dictionary_grouped)
X_train_grouped = train_bow_tweets(X_grouped, min_n_words=3)

Save to local:

In [14]:
df_grouped.to_csv(f'{directory}/data/model_data/tweets_tokenize_grouped_v2.csv')

Visualization of grouped tokens:

In [14]:
# Group by date as single documents, to process BOW
df.groupby(df.index).agg({'tokens': lambda x: list(chain(*x.to_list()))})[:2].style.set_properties(**{
    'text-align' : 'left',
    'white-space': 'wrap'
})